In [1]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from shapely.geometry import Point
from shapely.geometry import box
import os
import csv

# Marge the data. For now, use tolerance=0. (Do the tolerance=0.1 later)

In [11]:
# Load the agr and climate data in pandas
with open('./agr_climate_file_names.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    file_name_list = [row[0] for row in reader]

file_name_list.reverse()

print(file_name_list)
print(len(file_name_list))

['F162004.v4b_web.stable_lights.avg_vis', 'wc2.1_2.5m_tavg_12', 'wc2.1_2.5m_tavg_11', 'wc2.1_2.5m_tavg_10', 'wc2.1_2.5m_tavg_09', 'wc2.1_2.5m_tavg_08', 'wc2.1_2.5m_tavg_07', 'wc2.1_2.5m_tavg_06', 'wc2.1_2.5m_tavg_05', 'wc2.1_2.5m_tavg_04', 'wc2.1_2.5m_tavg_03', 'wc2.1_2.5m_tavg_02', 'wc2.1_2.5m_tavg_01', 'wc2.1_2.5m_bio_19', 'wc2.1_2.5m_bio_18', 'wc2.1_2.5m_bio_17', 'wc2.1_2.5m_bio_16', 'wc2.1_2.5m_bio_15', 'wc2.1_2.5m_bio_14', 'wc2.1_2.5m_bio_13', 'wc2.1_2.5m_bio_12', 'wc2.1_2.5m_bio_11', 'wc2.1_2.5m_bio_10', 'wc2.1_2.5m_bio_9', 'wc2.1_2.5m_bio_8', 'wc2.1_2.5m_bio_7', 'wc2.1_2.5m_bio_6', 'wc2.1_2.5m_bio_5', 'wc2.1_2.5m_bio_4', 'wc2.1_2.5m_bio_3', 'wc2.1_2.5m_bio_2', 'wc2.1_2.5m_bio_1', 'pre1500OptCaloriesNo0', 'pre1500encycle0', 'post1500OptCaloriesNo0', 'post1500encycle0', 'difencycle0', 'difen0']
38


## Merge the files using pandas.

In [5]:
merged_df = pd.read_csv(f'./agr_climate_csv/{file_name_list[0]}.csv')

for i in range(1,len(file_name_list)):
    df_a = pd.read_csv(f'./agr_climate_csv/{file_name_list[i]}.csv')

    merged_df = pd.merge(df_a, merged_df, on=['latitude', 'longitude', 'country_name'],how = 'inner')


cols = merged_df.columns.tolist()
cols = [cols[0], cols[1], cols[3], cols[2]]+ cols[4:] 
merged_df = merged_df[cols]

merged_df.to_csv('./merged_Agr_Cli_Lum_samples/marged_F162004_all_agr_cli_lum.csv', index =False)



## Load and check the merged file.

In [6]:
file = pd.read_csv('./merged_Agr_Cli_Lum_samples/marged_F162004_all_agr_cli_lum.csv')
file.head()

,latitude,longitude,country_name,difen0,difencycle0,post1500encycle0,post1500OptCaloriesNo0,pre1500encycle0,pre1500OptCaloriesNo0,wc2.1_2.5m_bio_1,...,wc2.1_2.5m_tavg_04,wc2.1_2.5m_tavg_05,wc2.1_2.5m_tavg_06,wc2.1_2.5m_tavg_07,wc2.1_2.5m_tavg_08,wc2.1_2.5m_tavg_09,wc2.1_2.5m_tavg_10,wc2.1_2.5m_tavg_11,wc2.1_2.5m_tavg_12,F162004.v4b_web.stable_lights.avg_vis
0,38.5,60.9,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1,38.5,61.3,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
2,38.5,61.7,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
3,38.5,63.0,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
4,38.5,63.4,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


## Check without lumosity data.

In [14]:
file_name_list.remove('F162004.v4b_web.stable_lights.avg_vis')
merged_df = pd.read_csv(f'./agr_climate_csv/{file_name_list[0]}.csv')

for i in range(1,len(file_name_list)):
    df_a = pd.read_csv(f'./agr_climate_csv/{file_name_list[i]}.csv')

    merged_df = pd.merge(df_a, merged_df, on=['latitude', 'longitude', 'country_name'],how = 'inner')


cols = merged_df.columns.tolist()
cols = [cols[0], cols[1], cols[3], cols[2]]+ cols[4:] 
merged_df = merged_df[cols]

merged_df.to_csv('./merged_Agr_Cli_Lum_samples/marged_all_agr_cli_no_lum.csv', index =False)



In [15]:
file_no_lum = pd.read_csv('./merged_Agr_Cli_Lum_samples/marged_all_agr_cli_no_lum.csv')
file_no_lum.head()

,latitude,longitude,country_name,difen0,difencycle0,post1500encycle0,post1500OptCaloriesNo0,pre1500encycle0,pre1500OptCaloriesNo0,wc2.1_2.5m_bio_1,...,wc2.1_2.5m_tavg_03,wc2.1_2.5m_tavg_04,wc2.1_2.5m_tavg_05,wc2.1_2.5m_tavg_06,wc2.1_2.5m_tavg_07,wc2.1_2.5m_tavg_08,wc2.1_2.5m_tavg_09,wc2.1_2.5m_tavg_10,wc2.1_2.5m_tavg_11,wc2.1_2.5m_tavg_12
0,38.5,60.5,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
1,38.5,60.9,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
2,38.5,61.3,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
3,38.5,61.7,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
4,38.5,62.1,Afghanistan,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-3.400000e+38,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38
